In [1]:
# Install packages
!pip install -q requests boto3 google-cloud-storage
!pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.6 MB/s eta 0:00:00


In [ ]:
# 2. Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Data Collection

In [ ]:
# Method-1: Web Scraping Wikipedia Source for CNSC
!python /content/drive/MyDrive/Colab\ Notebooks/Pipeline/1.\ Data\ Collection/wiki.py

Articles found: 30
Article limit reached. Stopping the process.

Total articles found and added to ZIP: 30
Total time taken: 134.34 seconds


In [ ]:
# # Method-2: Web Scraping Wikipedia Source for all wikipedia_source # You can put article limit in .py document
# !python /content/drive/MyDrive/Colab\ Notebooks/Pipeline/1.\ Data\ Collection/script_get_wiki_articles_w_key_words_canada_glossary.py

|██████████████████████████████████████████████████| 100.0% Complete - Elapsed time: 35.13 seconds

## 2. Q&A Generator

In [ ]:
# Q&A Generator and saved
!python /content/drive/MyDrive/Colab\ Notebooks/Pipeline/2.\ Pair\ Generator/generator.py

In [ ]:
import gdown
# Google Drive file ID
file_id = '1yl9K1M_Ey86DCU26jHX69v2GQhoURIcf'
# Local file path where the downloaded file will be saved
output_path = 'qa_pairs.json'
# Download the file from Google Drive
gdown.download(f'https://drive.google.com/uc?id={file_id}', output_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1yl9K1M_Ey86DCU26jHX69v2GQhoURIcf
To: /content/qa_pairs.json
100%|██████████| 43.0k/43.0k [00:00<00:00, 40.9MB/s]


'qa_pairs.json'

In [ ]:
# # Load your data
# import json
# with open("qa_pairs.json") as json_file:
#     data = json.load(json_file)
# from pprint import pprint

# pprint(data["questions"][1], sort_dicts=False)

{'question': 'What is the role of the CNSC?',
 'answer': 'The Participant Funding Program allows the public, Indigenous '
           'groups, and other stakeholders to request funding from the CNSC to '
           'participate in its regulatory processes.'}


# 3. Model Fine-Tuning and Saving

In [ ]:
!python /content/drive/MyDrive/Colab\ Notebooks/Pipeline/3.\ Model/model.py

In [ ]:
# Specify the file ID
file_id = "1mSEhq47G7tLk60olxqv9wfwCDlBHRlYg"
destination = "content/"
# Download the file
!gdown --id {file_id} -O {destination}
print(f"File successfully downloaded: {destination}")

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1mSEhq47G7tLk60olxqv9wfwCDlBHRlYg
From (redirected): https://drive.google.com/uc?id=1mSEhq47G7tLk60olxqv9wfwCDlBHRlYg&confirm=t&uuid=6090a4c4-6a71-4e70-8f02-3fc3ee91b3ea
To: /content/content/trained-model.zip
100% 27.6M/27.6M [00:00<00:00, 138MB/s]
File successfully downloaded: content/


# 4. Evaluation

In [ ]:
# Install necessary libraries
!pip install transformers
!pip install nltk

# Clone the question generator repository
!git clone https://github.com/erindakapllani/question_generator/
%cd question_generator/

# Import necessary libraries
import torch
from questiongenerator import QuestionGenerator, print_qa
from nltk import sent_tokenize

# Ensure GPU is being used
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
assert device == torch.device('cuda'), "Not using CUDA. Set: Runtime > Change runtime type > Hardware Accelerator: GPU"

# Initialize the Question Generator
qg = QuestionGenerator()

import os

# List all files in the 'articles' folder
article_files = os.listdir('articles')
article_contents = []
# Read contents of each file
for file_name in article_files:
    file_path = os.path.join('articles', file_name)
    with open(file_path, 'r') as file:
        article_contents.append(file.read())

# Combine all article contents into a single string
combined_article = "\n".join(article_contents)


# Generate questions
qa_list = qg.generate(combined_article, num_questions=37, answer_style='all')
print_qa(qa_list)

Cloning into 'question_generator'...
remote: Enumerating objects: 273, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 273 (delta 87), reused 24 (delta 24), pack-reused 86
Receiving objects: 100% (273/273), 19.15 MiB | 27.82 MiB/s, done.
Resolving deltas: 100% (100/100), done.
/content/question_generator


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Generating questions...

